In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\baver\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import cv2
from google.colab import files
from ultralytics import YOLO
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#Upload video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

#Open video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("Video file could not be opened! Please check the file.")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Load YOLO model
model = YOLO('yolov8n.pt')

#Video save settings
out = cv2.VideoWriter('processed_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
if not out.isOpened():
    raise ValueError("VideoWriter could not be initialized!")

#Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video prcessed successfully.")
        break

    #Object detection
    results = model.predict(frame, stream=True)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            confidence = box.conf[0].item()
            cls = box.cls[0].item()
            label = f"{model.names[int(cls)]} {confidence:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

#Source release
cap.release()
out.release()

In [ ]:
# Question and options
question = widgets.RadioButtons(
    options=['Answer', 'Yes', 'No'],
    description='Do you wanna download processed video file?',
    disabled=False
)

# Answer: Yes -> Download the video file No -> Do not download the video file
def on_button_click(change):
    if question.value == 'Yes':
        #Download the video file
        files.download("processed_video.mp4")
    else:
        #Video file is not downloaded
        print("Video file is not downloaded.")

#Show the widget
question.observe(on_button_click, names='value')
display(question)

#!!!If video file is too big then you should have to wait more after clicking on 'Yes' option to download the video file.!!!